In [13]:
import numpy as np
import pandas as pd
import json
import joblib
df = pd.read_csv('https://raw.githubusercontent.com/pplonski/datasets-for-start/master/adult/data.csv', skipinitialspace=True)
x_cols = [c for c in df.columns if c != 'income']
# set input matrix and target column
X = df[x_cols]
y = df['income']
# show first rows of data
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [15]:
cat=[]
for i in X.columns:
    if type(X[i][0])==str:
        cat.append(i)
print(cat)

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


In [16]:
train_mode = dict(X_train.mode().iloc[0])
X_train = X_train.fillna(train_mode)
print(train_mode)

{'age': 36, 'workclass': 'Private', 'fnlwgt': 164190, 'education': 'HS-grad', 'education-num': 9, 'marital-status': 'Married-civ-spouse', 'occupation': 'Craft-repair', 'relationship': 'Husband', 'race': 'White', 'sex': 'Male', 'capital-gain': 0, 'capital-loss': 0, 'hours-per-week': 40, 'native-country': 'United-States'}


In [17]:
from sklearn.preprocessing import LabelEncoder
encoders = {}
for i in cat:
    le = LabelEncoder()
    X_train[i] = le.fit_transform(X_train[i])
    #X_test[i] = le.transform(X_test[i])
    encoders[i] = le


In [18]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, criterion='entropy', random_state=10)
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', random_state=10)

In [19]:
joblib.dump(train_mode, './train_mode.joblib', compress=True)
joblib.dump(encoders, './encoders.joblib', compress=True)
joblib.dump(rfc, './rfc.joblib', compress=True)
print(encoders)
print(train_mode)
print(rfc)
print(encoders['workclass'].transform(['Private']))

{'workclass': LabelEncoder(), 'education': LabelEncoder(), 'marital-status': LabelEncoder(), 'occupation': LabelEncoder(), 'relationship': LabelEncoder(), 'race': LabelEncoder(), 'sex': LabelEncoder(), 'native-country': LabelEncoder()}
{'age': 36, 'workclass': 'Private', 'fnlwgt': 164190, 'education': 'HS-grad', 'education-num': 9, 'marital-status': 'Married-civ-spouse', 'occupation': 'Craft-repair', 'relationship': 'Husband', 'race': 'White', 'sex': 'Male', 'capital-gain': 0, 'capital-loss': 0, 'hours-per-week': 40, 'native-country': 'United-States'}
RandomForestClassifier(criterion='entropy', random_state=10)
[3]


In [20]:
rfi = RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=0)

In [21]:
rfi.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=500, random_state=0)

In [24]:
joblib.dump(rfi, './rfi.joblib', compress=True)

['./rfi.joblib']

In [25]:
import requests
import json 

In [48]:
for i in range(300):
    input_data = dict(X_test.iloc[i])
    target = y_test.iloc[i]
    r = requests.post("http://127.0.0.1:8000/api/v1/income_classifier/predict?status=ab_testing", input_data)
    #print(type(r))
    print(i)
    response = r.json()
    #print(type(response))
    requests.put("http://127.0.0.1:8000/api/v1/mlrequests/{}".format(response["request_id"]), {"feedback": target})


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27